In [22]:
import pandas as pd
import sqlite3
import json
import os
import re
from typing import Any
import requests
from bs4 import BeautifulSoup, ResultSet
from copy import deepcopy
from IPython.display import HTML, display
from pathlib import Path
from pinecone import ServerlessSpec
from pinecone.grpc import PineconeGRPC
from langchain.document_loaders import TextLoader
from llama_index.core.indices.vector_store.base import VectorStoreIndex
from llama_index.core.readers import download_loader
from llama_index.core.ingestion.pipeline import IngestionPipeline
from llama_index.core.node_parser import SemanticSplitterNodeParser
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.schema import Document, TransformComponent
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.readers.file import PDFReader
from llama_index.readers.file import CSVReader
from llama_index.vector_stores.pinecone import PineconeVectorStore
from unstructured.partition.pdf import partition_pdf
from openai import OpenAI




In [38]:
!pip install -qU \
    "pinecone-client[grpc]"==3.2.2 \
    "unstructured[pdf]"==0.12.4 \
    langchain==0.1.9 \
    llama-index==0.10.23 \
    llama-index-vector-stores-pinecone==0.1.4 \
    pillow==10.0.0 \
    poppler-utils==0.1.0 \
    pytesseract==0.3.10

## Load + Clean +SQL Queries + Question and Answers

In [23]:
#Load the CSV file
huddf = pd.read_csv('data/hud_insured_multifamily.csv')

C:\Users\mdalt\AppData\Local\Temp\ipykernel_45856\3972625611.py:2: DtypeWarning: Columns (2,19,22,35,36,41,89,90,91,92,95,103,109,111,121,123,125,126,127,164,169,170,176,205,209,217,274) have mixed types. Specify dtype option on import or set low_memory=False.
  huddf = pd.read_csv('data/hud_insured_multifamily.csv')


In [27]:
# Data cleaning
#huddf = huddf.drop(columns=["PROJECT_MANAGER_NAME_TEXT", "AUTOMATIC_GEOCODE_IND"])
huddf = huddf[["STD_CITY","PROPERTY_ID","PROPERTY_CATEGORY_NAME","PROPERTY_NAME_TEXT","ORIGINAL_LOAN_AMOUNT","LOAN_MATURITY_DATE","OPIIS_SIZE_CATEGORY"]]
#huddf = huddf.head(100)
huddf.to_csv('data/hud_insured_multifamily_cle_HUMAN.csv', index=False)


In [28]:
# Connect to a SQLite database
conn = sqlite3.connect('database.db')


In [29]:
# TABLE command
create_table_command = """
CREATE TABLE IF NOT EXISTS HUD_Insured_Multifamily1 (
    STD_CITY,
    PROPERTY_ID,
    PROPERTY_CATEGORY_NAME,
    Property_NAME_TEXT,
    ORIGINAL_LOAN_AMOUNT,
    LOAN_MATURITY_DATE,
    OPIIS_SIZE_CATEGORY


)
"""
conn.execute(create_table_command)
conn.commit()

In [30]:
# Function to insert values from CSV to the table
def insert_values_from_csv(conn, csv_file_path):
    df = pd.read_csv(csv_file_path)
    df.to_sql('HUD_Insured_Multifamily1', conn, if_exists='replace', index=False)


In [31]:
# Insert values into the table from a CSV file
insert_values_from_csv(conn, 'data/hud_insured_multifamily_cle_HUMAN.csv')

In [32]:
# Query the table and print the result
table = pd.read_sql_query("SELECT * FROM HUD_Insured_Multifamily1", conn)
print(table)

                           STD_CITY  PROPERTY_ID PROPERTY_CATEGORY_NAME  \
0      Pennington                    800225589.0   Insured-Unsubsidized   
1      White Settlement              800225933.0   Insured-Unsubsidized   
2      Carlinville                   800225983.0   Insured-Unsubsidized   
3      Grand Coulee                  800226573.0   Insured-Unsubsidized   
4      Vidalia                       800226611.0   Insured-Unsubsidized   
...                             ...          ...                    ...   
16519  Maplewood                     800254374.0   Insured-Unsubsidized   
16520  Garland                       800255132.0   Insured-Unsubsidized   
16521  Pleasant Hill                 800255134.0   Insured-Unsubsidized   
16522  Grasonville                   800255144.0   Insured-Unsubsidized   
16523  Southgate                     800255510.0   Insured-Unsubsidized   

                               PROPERTY_NAME_TEXT  ORIGINAL_LOAN_AMOUNT  \
0                       

In [33]:
# Prepare for answering the questions
answers = []

In [34]:
# What city has the most properties?
cursor = conn.execute("SELECT STD_CITY, COUNT(*) FROM HUD_Insured_Multifamily1 GROUP BY STD_CITY ORDER BY COUNT(*) DESC LIMIT 1")
answers.append(f"City with most properties: {cursor.fetchone()}")


In [35]:
# How many property names start with the letter 'H'
cursor = conn.execute("SELECT COUNT(*) FROM HUD_Insured_Multifamily1 WHERE Property_NAME_TEXT LIKE 'H%'")
answers.append(f"Properties starting with 'H': {cursor.fetchone()[0]}")


In [36]:
# What % of the properties in the property category is 'Insured-Unsubsidized'
cursor = conn.execute("SELECT COUNT(*) FROM HUD_Insured_Multifamily1 WHERE PROPERTY_CATEGORY_NAME = 'Insured-Unsubsidized'")
total_count = cursor.fetchone()[0]
cursor = conn.execute("SELECT COUNT(*) FROM HUD_Insured_Multifamily1")
overall_count = cursor.fetchone()[0]
percentage = (total_count / overall_count) * 100 if overall_count > 0 else 0
answers.append(f"Percentage of 'Insured-Unsubsidized': {percentage:.2f}%")

In [37]:
# How many unique property id are there in this data set
cursor = conn.execute("SELECT COUNT(DISTINCT PROPERTY_ID) FROM HUD_Insured_Multifamily1")
answers.append(f"Unique property IDs: {cursor.fetchone()[0]}")

In [38]:
# What were the three largest refinaces
cursor = conn.execute("SELECT PROPERTY_NAME_TEXT, ORIGINAL_LOAN_AMOUNT FROM HUD_Insured_Multifamily1 ORDER BY ORIGINAL_LOAN_AMOUNT DESC LIMIT 3")
answers.append(f"Three largest refinances: {cursor.fetchall()}")

In [39]:
#Does the size of a refinanace correlate with the capacity of a multi family ?
cursor = conn.execute("""
SELECT OPIIS_SIZE_CATEGORY, AVG(ORIGINAL_LOAN_AMOUNT) AS Average_Loan_Amount
FROM HUD_Insured_Multifamily1
GROUP BY OPIIS_SIZE_CATEGORY
ORDER BY Average_Loan_Amount DESC
""")
answers.append(f"Refinance size and multifamily capacity correlation: {cursor.fetchall()}")

In [40]:
# What are distnict categories of finacing present in the data and what do they mean?
cursor = conn.execute("SELECT DISTINCT PROPERTY_CATEGORY_NAME FROM HUD_Insured_Multifamily1")
categories = cursor.fetchall()
answers.append(f"Distinct financing categories: {categories}")

In [41]:
# Close the connection
conn.close()

In [42]:
# Combine questions and answers into a DataFrame
qa_df = pd.DataFrame({
    "Questions": [
        "What city has the most properties?",
        "How many property names start with the letter 'H'",
        "What % of the properties in the property category is 'Insured-Unsubsidized'",
        "How many unique property IDs are there in this data set",
        "What were the three largest refinaces?",
        "Does the size of a refinanace correlate with the capacity of a multi family ?",
        "What are distnict categories of finacing present in the data and what do they mean?"
    ],
    "Answers": answers
})

In [43]:
# Export the DataFrame to a CSV file
qa_df.to_csv(r'data/Questions_and_Answers_HUMAN1.csv', index=False)

In [102]:
print(qa_df)

                                           Questions  \
0                 What city has the most properties?   
1  How many property names start with the letter 'H'   
2  What % of the properties in the property categ...   
3  How many unique property IDs are there in this...   
4             What were the three largest refinaces?   
5  Does the size of a refinanace correlate with t...   
6  What are distnict categories of finacing prese...   

                                             Answers  
0  City with most properties: ('Jacksonville     ...  
1                    Properties starting with 'H': 3  
2      Percentage of 'Insured-Unsubsidized': 100.00%  
3                            Unique property IDs: 88  
4  Three largest refinances: [('Capital Health Sy...  
5  Refinance size and multifamily capacity correl...  
6  Distinct financing categories: [('Insured-Unsu...  


In [103]:
# Read in your CSV

loader = CSVReader()
path = Path('data/hud_insured_multifamily_cle.csv')
ctrl_docs = loader.load_data(file=path)


In [104]:
print(ctrl_docs)

[Document(id_='b6e25b0f-e6f1-47f7-8a0c-c207d815ebe8', embedding=None, metadata={'filename': 'hud_insured_multifamily_cle.csv', 'extension': '.csv'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="STD_CITY, PROPERTY_ID, PROPERTY_CATEGORY_NAME, PROPERTY_NAME_TEXT, ORIGINAL_LOAN_AMOUNT, LOAN_MATURITY_DATE, OPIIS_SIZE_CATEGORY\nPennington                  , 800225589, Insured-Unsubsidized, Capital Health System, 755874500, 1/1/2037 0:00, \nWhite Settlement            , 800225933, Insured-Unsubsidized, Oak Timbers - White Settlement Apartments, 4000000, 3/1/2025 0:00, High-Medium\nCarlinville                 , 800225983, Insured-Unsubsidized, Carlinville, 19250000, 5/1/2035 0:00, \nGrand Coulee                , 800226573, Insured-Unsubsidized, Coulee Community Hospital, 18864200, 10/1/2041 0:00, \nVidalia                     , 800226611, Insured-Unsubsidized, Meadows Regional Medical Center, 74435000, 5/1/2036 0:00, \nDallas                      , 80

## PINECONE and OPENAI setup

In [105]:
os.environ['PINECONE_API_KEY'] = "f75a7c27-bf4d-4982-89bc-de3a6054715c"  
pinecone_api_key = os.getenv("PINECONE_API_KEY")

# Initialize connection to Pinecone
pc = PineconeGRPC(api_key=pinecone_api_key)
index_name = "structured-data-example"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        index_name,
        dimension=1536,  # Dimensions match encoder (embedder/vectorizer) you will use downstream, ada-002 from OpenAI.
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

# Initialize your index
pinecone_index = pc.Index(index_name)

In [106]:
# Confirm creation of your index & that (if new) it has no vectors in it yet.
pinecone_index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'control': {'vector_count': 10}},
 'total_vector_count': 10}

In [107]:
# If for any reason you want to delete your Pinecone index and start over, execute this code:
# pc.delete_index(index_name)

In [108]:
# Set/Get your OpenAI API Key

os.environ['OPENAI_API_KEY'] = "sk-proj-HsdNJgnkUhHogwB6B4eQT3BlbkFJsMUwVvP89qOrRYCAXg3v"  
openai_api_key = os.getenv("OPENAI_API_KEY")

In [109]:
# Declare embedding model you will use throughout notebook:
# OpenAI's ada-002 text embedding modal is the model you will use both for Node parsing and for vectorization of PDF contents
EMBED_MODEL = OpenAIEmbedding(api_key=openai_api_key)

In [110]:
# You will need to re-define Pinecone as a LlamaIndex PineconeVectorStore obj when you add namespaces, so build a
# function to help you do that:
def initialize_vector_store(index: PineconeGRPC, namespace: str) -> PineconeVectorStore:
    """
    Initialize Pinecone index as a VectorStore obj.

    :param index: Pinecone serverless index.
    :param namespace: Namespace constraint you want on your queries, indexing operations, etc. when using this vector store.
    :return: PineconeVectorStore obj.
    """
    return PineconeVectorStore(pinecone_index=index, namespace=namespace)

In [111]:
def run_indexing_pipeline(vector_store, documents, embed_model=EMBED_MODEL):
    # Define pipeline stages
    pipeline = IngestionPipeline(
        transformations=[
            # CleanTextForOpenAI(),  # Clean doc text
            SemanticSplitterNodeParser(
                buffer_size=1,
                breakpoint_percentile_threshold=95,
                embed_model=embed_model,
                ),
            embed_model,  # Vectorize nodes
            ],
       
        vector_store=vector_store # Index into Pinecone
        )

    # Run documents through pipeline
    return pipeline.run(documents=documents)

In [112]:


# Declare namespace you will put your first batch of vectors into:
ctrl_namespace = 'control'

# Initialize vector store w/control namespace
ctrl_vector_store = initialize_vector_store(pinecone_index, ctrl_namespace)

# Run pipeline
output = run_indexing_pipeline(ctrl_vector_store, ctrl_docs)



Upserted vectors:   0%|          | 0/2 [00:00<?, ?it/s]

In [113]:
# Confirm your docs made it to the index, in the right namespace
pinecone_index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'control': {'vector_count': 10}},
 'total_vector_count': 10}

In [114]:
QUERIES = [
    "What city has the most properties?",
    "How many property names start with the letter 'H'",
    "What % of the properties in the property category is 'Insured-Unsubsidized'",
    "How many unique property IDs are there in this data set",
    "What were the three largest refinaces?",
    "Does the size of a refinanace correlate with the capacity of a multi family ?",
    "What are distnict categories of finacing present in the data and what do they mean?",
    
]

ANSWERS = [
    "City with most properties: ('Jacksonville', 3)",
    " Properties starting with 'H': 3",
    " Percentage of 'Insured-Unsubsidized': 100.00%",
    " Unique property IDs: 100",
    """Three largest refinances: [('Capital Health System', 755874500.0), ('UNIVERSITY OF NEW MEXICO HOSPITAL', 320000000.0), ("ST. LUKE'S ROOSEVELT HOSPITAL", 319555000.0)]""",
    "Refinance size and multifamily capacity correlation: [(None, 36928028.652173914), ('Large', 11553500.0), ('High-Medium', 3567451.4), ('Low-Medium', 1672000.0)]",
    "Distinct financing categories: [('Insured-Unsubsidized',)]",
]



In [115]:
def run_rag_pipeline(vector_store, queries, k=5, filters=None):
    """
    Send queries to an LLM, having it take context from a vector store (and namespace).

    :param vector_store: Your Pinecone vector store.
    :param queries: The queries you want to ask your LLM.
    :param k: The number of results you want retrieved as context from your Pinecone index.
    :param filters: Option to add metadata filters to request if desired.
    :return: Tuple of responses from your LLM.
    """

    # Instantiate VectorStoreIndex object from our vector_store object
    vector_index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

    if not filters:
        retriever = VectorIndexRetriever(index=vector_index, similarity_top_k=k, namespace=vector_store.namespace)
    else:
        retriever = VectorIndexRetriever(index=vector_index, similarity_top_k=k, namespace=vector_store.namespace, filters=filters)

    # Query engine
    query_engine = RetrieverQueryEngine(retriever=retriever)

    # Pass our 4 test queries
    responses = ()
    for i in queries:
        response = query_engine.query(i).response
        responses += (response, )

    return responses

In [116]:
one_ctrl, two_ctrl, three_ctrl, four_ctrl, five_ctrl, six_ctrl, seven_ctrl = run_rag_pipeline(ctrl_vector_store, QUERIES)


In [117]:
print(f"One: {one_ctrl}\n-----\nTwo: {two_ctrl}\n-----\nThree: {three_ctrl}\n-----\nFour: {four_ctrl}\n-----\nFive: {five_ctrl}\n-----\nSix: {six_ctrl}\n-----\nSeven: {seven_ctrl}\n")

One: Jacksonville
-----
Two: 6
-----
Three: 100%
-----
Four: There are 47 unique property IDs in this data set.
-----
Five: The three largest refinances were for the properties associated with the following cities: Albuquerque, Buffalo, and New York.
-----
Six: The size of a refinanace does not necessarily correlate with the capacity of a multi family.
-----
Seven: The distinct categories of financing present in the data are "Insured-Unsubsidized." This category typically refers to properties that have obtained insurance for their loans without receiving any additional subsidies from the government.

